In [45]:
import statsmodels as sm
import linearmodels
import pandas as pd
import numpy as np

path_ana_red = 'copy_master_data/Anagrafica_ClubQ8_red.csv'
path_premi_red = 'copy_master_data/Premi_ClubQ8_red.csv'
path_rif_red = 'copy_master_data/Rifornimenti_Carburante_ClubQ8_red.csv'

df1 = pd.read_csv(path_ana_red, sep=';', decimal = ',')
df1.head() #OK, works as expected and converts the decimals

df2 = pd.read_csv(path_premi_red, sep=';', decimal = ',')
df3 = pd.read_csv(path_rif_red, sep=';', decimal = ',')

In [46]:
df1.drop(['Unnamed: 0'], axis = 1, inplace= True)
df2.drop(['Unnamed: 0'], axis = 1, inplace= True)
df3.drop(['Unnamed: 0'], axis = 1, inplace= True)

df1.head()

,COD_PAN_DA_POS,SEX,DATA_NASCITA,REGIONE,PROVINCIA,COMUNE,TIPO_CARTA,DATA_BATTESIMO,SALDO_PUNTI
0,7071161603018922195,M,1951-09-14 00:00:00,LAZIO,VITERBO,SUTRI,STARQ8_FISICA,2015-01-08 12:32:58,511.0
1,7071161603005992565,F,1988-09-15 00:00:00,NaN,NaN,NaN,STARQ8_FISICA,2020-07-10 12:22:35,379.0
2,7071161603025867243,F,1943-09-09 00:00:00,LAZIO,ROMA,ROMA,STARQ8_FISICA,2020-07-10 18:07:14,108.0
3,7071161603021997861,M,1974-03-01 00:00:00,SICILIA,MESSINA,MESSINA,STARQ8_FISICA,2020-07-10 18:08:21,100.0
4,7071161603025554049,M,1963-11-22 00:00:00,CAMPANIA,SALERNO,EBOLI,STARQ8_FISICA,2020-07-10 18:09:27,284.0


In [47]:
df2.columns

Index(['COD_PAN_DA_POS', 'DATA_OPERAZIONE', 'LUOGO_PRENOTAZIONE_PREMIO',
       'CATEGORIA', 'RAGGRUPPAMENTO_MERCEOLOGICO', 'DESCRIZIONE',
       'PUNTI_RICHIESTI', 'CONTRIBUTO_CLIENTE_CON_IVA'],
      dtype='object')

In [48]:
#df4 = pd.DataFrame(columns = 'Test')
df1['DATA_BATTESIMO'] = df1['DATA_BATTESIMO'].astype('datetime64')
df1['DATA_BATTESIMO'] = df1['DATA_BATTESIMO'].dt.date

df2['DATA_OPERAZIONE'] = df2['DATA_OPERAZIONE'].astype('datetime64')
df2['DATA_OPERAZIONE'] = df2['DATA_OPERAZIONE'].dt.date
df2.head()

,COD_PAN_DA_POS,DATA_OPERAZIONE,LUOGO_PRENOTAZIONE_PREMIO,CATEGORIA,RAGGRUPPAMENTO_MERCEOLOGICO,DESCRIZIONE,PUNTI_RICHIESTI,CONTRIBUTO_CLIENTE_CON_IVA
0,7071161603670014604,2020-09-01,ONLINE,PREMIO FISICO,PER LA TUA CASA,CONSEGNA CODICE GIFT CARDS 20E AMAZON,1800,NaN
1,7071161603670014604,2020-09-01,ONLINE,PREMIO FISICO,PER LA TUA CASA,CONSEGNA CODICE GIFT CARDS 20E AMAZON,1800,NaN
2,7071161603670014604,2020-09-01,ONLINE,PREMIO FISICO,PER LA TUA CASA,CONSEGNA CODICE GIFT CARDS 20E AMAZON,1800,NaN
3,7071161603670014604,2020-09-01,ONLINE,PREMIO FISICO,PER LA TUA CASA,CONSEGNA CODICE GIFT CARDS 20E AMAZON,1800,NaN
4,7071161603014051338,2020-09-01,PUNTO VENDITA,PREMIO FISICO,PER TE,PRENOTAZIONE PORTA CARTE CREDITO SPARCO,450,0.0


In [82]:
df3['DATA_OPERAZIONE'] = df3['DATA_OPERAZIONE'].astype('datetime64')
df3['DATA_OPERAZIONE'] = df3['DATA_OPERAZIONE'].dt.date
df3.head()

df2.sort_values(by=['DATA_OPERAZIONE'], inplace=True, ascending=False)
df2.drop_duplicates('COD_PAN_DA_POS', keep='first', inplace=True)

df4 = df2.merge(df3, how='inner', on='COD_PAN_DA_POS')
print(df4.columns)
df4 = df4.iloc[:,[0,1,9,12]]

cols = df4.columns.tolist()
print(cols)
new_cols = [cols[2], cols[3], cols[0], cols[1]]
df4 = df4[new_cols]
df4 = df4.rename(columns={'DATA_OPERAZIONE_y': 'Date of gas purchase', 'LITRI': 'Gas Purchase in liters', 'COD_PAN_DA_POS': 'Unique Customer Identifier', 'DATA_OPERAZIONE_x': 'Date of first prize claim'})
#delta = df4.iloc[0,0] - df4.iloc[0,3]

#df1['Score_diff'] = df1['score1'].sub(df1['score2'], axis = 0)
df4['Date difference'] = df4['Date of gas purchase'].sub(df4['Date of first prize claim'], axis = 0)
df4

Index(['COD_PAN_DA_POS', 'DATA_OPERAZIONE_x', 'LUOGO_PRENOTAZIONE_PREMIO',
       'CATEGORIA', 'RAGGRUPPAMENTO_MERCEOLOGICO', 'DESCRIZIONE',
       'PUNTI_RICHIESTI', 'CONTRIBUTO_CLIENTE_CON_IVA', 'COD_PV',
       'DATA_OPERAZIONE_y', 'PRODOTTO', 'MODALITA_VENDITA', 'LITRI',
       'PUNTI_CARBURANTE'],
      dtype='object')
['COD_PAN_DA_POS', 'DATA_OPERAZIONE_x', 'DATA_OPERAZIONE_y', 'LITRI']


,Date of gas purchase,Gas Purchase in liters,Unique Customer Identifier,Date of first prize claim,Date difference
0,2021-03-01,30,7071161603670009711,2020-09-01,181 days
1,2021-03-01,25,7071161603012857264,2020-09-01,181 days
2,2021-03-01,50,7071161603670311075,2020-09-01,181 days
3,2021-03-01,50,7071161603670676287,2020-09-01,181 days
